# Video Indexer Python Sample

This sample will be using a simple `requests` library wrapper around the Microsoft Video Indexer API based on an OSS project:  https://github.com/bklim5/python_video_indexer_lib.

Resources:

* <a href="https://docs.microsoft.com/en-us/azure/media-services/video-indexer/" target="_blank">Video Indexer Docs</a>
* <a href="https://api-portal.videoindexer.ai" target="_blank">Video Indexer Developer Portal</a>
* <a href="https://api-portal.videoindexer.ai/docs/services" target="_blank">Video Indexer API Reference</a>
* <a href="http://2.python-requests.org/en/latest/" target="_blank">Python Requests library</a>

## Setup and imports

In [ ]:
import os
import requests

Download a public video of fish swimming, locally:

In [ ]:
! curl -O https://github.com/Azadehkhojandi/computer-vision-fish-frame-proposal/raw/master/videos/video1.mp4

In [ ]:
# A local file (we can also send URLs such as private blob storage links)
video_file = 'video1.mp4'

## Initialize with Video Indexer (VI) account info

SUBSCRIPTION_KEY can be found at https://api-portal.videoindexer.ai/developer (go to your user name --> Profile) and LOCATION & ACCOUNT_ID can be found at https://www.videoindexer.ai/settings/account.  If this is a trial account, use "trial" as LOCATION.

In [ ]:
CONFIG = {
    'SUBSCRIPTION_KEY': '<VI subscription key>',
    'LOCATION': '<azure region of VI e.g. westus>',
    'ACCOUNT_ID': '<VI account ID>'
}

In [ ]:
headers = {
    'Ocp-Apim-Subscription-Key': CONFIG['SUBSCRIPTION_KEY']
}

params = {
    'allowEdit': 'True'
}

access_token_req = requests.get(
    'https://api.videoindexer.ai/auth/{loc}/Accounts/{acc_id}/AccessToken'.format(
        loc=CONFIG['LOCATION'],
        acc_id=CONFIG['ACCOUNT_ID']
    ),
    params=params,
    headers=headers
)

access_token = access_token_req.text[1:-1]
print('Access Token: {}'.format(access_token))

## Upload a video for indexing

More information on parameters can be found at https://docs.microsoft.com/en-us/azure/media-services/video-indexer/upload-index-videos.

In [ ]:
def upload_to_video_indexer(access_token, input_filename, video_name='', video_language='English'):
    """Upload a video file to a Video Indexer account"""
    print('Uploading video to video indexer...')
    params = {
        'streamingPreset': 'Default',
        'indexingPreset': 'Default',
        'language': video_language,
        'name': video_name,
        'accessToken': access_token
    }

    files = {
        'file': open(input_filename, 'rb')
    }

    upload_video_req = requests.post(
        'https://api.videoindexer.ai/{loc}/Accounts/{acc_id}/Videos'.format(
            loc=CONFIG['LOCATION'],
            acc_id=CONFIG['ACCOUNT_ID']
        ),
        params=params,
        files=files
    )

    if upload_video_req.status_code != 200:
        print('Error uploading video to video indexer: {}'.format(upload_video_req.json()))
        raise Exception('Error uploading video to video indexer')

    response = upload_video_req.json()
    return response['id']

In [ ]:
video_id = upload_to_video_indexer(access_token, video_file, video_name='australian-fish-sample')

## Analyze video with Video Indexer

The `get_video_info` below will indicate the processing status by video id.

In [ ]:
def get_video_info(access_token, video_id, video_language='English'):
    """Indicate the processing status"""
    params = {
        'accessToken': access_token,
        'language': video_language
    }
    print('Getting video info for: {}'.format(video_id))

    get_video_info_req = requests.get(
        'https://api.videoindexer.ai/{loc}/Accounts/{acc_id}/Videos/{video_id}/Index'.format(
            loc=CONFIG['LOCATION'],
            acc_id=CONFIG['ACCOUNT_ID'],
            video_id=video_id
        ),
        params=params
    )
    response = get_video_info_req.json()

    if response['state'] == 'Processing':
        print('Video still processing, current status: {}'.format(
            response['videos'][0]['processingProgress']))

    return response

In [ ]:
response_video_info = get_video_info(access_token, video_id)

## Extract the summary

In [ ]:
def extract_text_summary_from_video_indexer_info(info):
    """
    Extract text features like keywords, sentiment and
    transcript
    """
    return {
        'durationInSeconds': info['durationInSeconds'],
        'numberOfKeywords': len(info['summarizedInsights'].get('keywords', [])),
        'keywords': info['summarizedInsights'].get('keywords', []),
        'sumOfWordCount': sum(info['summarizedInsights']['statistics']['speakerWordCount'].values()),
        'sentimentSeenDurationRatio': {
            x['sentimentKey']: x['seenDurationRatio'] for x in info['summarizedInsights']['sentiments']
        },
        'sentimentScore': {
            x['sentimentType']: x['averageScore'] for x in info['videos'][0]['insights'].get('sentiments', [])
        },
        'transcript': [
            {
                'confidence': x['confidence'],
                'text': x['text'],
                'textLength': len(x['text'].split()),
                'confidencePerText': x['confidence'] * len(x['text'].split())
            } for x in info['videos'][0]['insights'].get('transcript', [])
        ]
    }

In [ ]:
extract_text_summary_from_video_indexer_info(response_video_info)

In [ ]:
def extract_vision_summary_from_video_indexer_info(info):
    """Extract visual detected object names"""
    results = {}
    for i, label in enumerate(info['summarizedInsights']['labels']):
        if 'detected_labels' in results:
            results['detected_labels'].append(label['name'])
        else:
            results['detected_labels'] = [label['name']]
    return results

In [ ]:
extract_vision_summary_from_video_indexer_info(response_video_info)

## Appendix A:  Get a video into Blob Storage

Run a script such as:  https://github.com/michhar/azure-and-ml-utils/blob/master/azure/upload_to_blob_storage.py or, if running the notebook locally, the following snippet with do the trick.

In [ ]:
import os
from azure.storage.blob import BlockBlobService, PublicAccess
import glob

# Create the BlockBlockService that is used to call the Blob service for the storage account
block_blob_service = BlockBlobService(account_name=args.account, account_key=args.key) 

# Create a container
container_name = args.container
block_blob_service.create_container(container_name) 

# Set the permission so the blobs are public.
block_blob_service.set_container_acl(container_name, public_access=PublicAccess.Container)

for filename in glob.iglob(os.path.join(args.directory, '**', '*.'+args.suffix), recursive=True):
    print('Uploading ', filename)
    # Upload the created file, use local_file_name for the blob name
    block_blob_service.create_blob_from_path(container_name, filename, filename)

# Check that the files uploaded correctly to blob
generator = block_blob_service.list_blobs(container_name)
for blob in generator:
    print("Blob name in Azure: " + blob.name)

## Get a video from Blob Storage

In [ ]:
from azure.storage.blob.baseblobservice import BaseBlobService
from azure.storage.blob import BlockBlobService, BlobPermissions
from datetime import datetime, timedelta

account_name = '<your account name>'
account_key = '<your account key>'
container_name = '<your container name>'
blob_name = '<your blob name>'
service = BaseBlobService(account_name=account_name, account_key=account_key)

token = service.generate_blob_shared_access_signature(container_name, blob_name, BlobPermissions.READ, datetime.utcnow() + timedelta(hours=1),)
blobUrlWithSas = f"https://{account_name}.blob.core.windows.net/{container_name}/{blob_name}?{token}"